In [14]:
%load_ext autoreload 
%autoreload 2

import sys
import os 

from pathlib import Path

PROJECT_ROOT = str(Path().resolve().parent)
sys.path.append(PROJECT_ROOT)

Модель использует следующие дневные данные для генерации признаков:
- price - значение индекс MCFTRR
- bonds10y - значение дох-ти к погашению RUGBICP10Y
- ruonia - значение ставки RUONIA
- top3_mean_vol - средний дневной объем торгов Сбера, Газпрома, Лукойла
- bonds10y_ruonia_delta - разница между ruonia и bonds10y
- price_vol_adj - динамика изменения индекса MCFTRR с поправкой на top3_mean_vol
- brent - цена на нефть Brent
- gold - цена золота по данным Центрального Банка РФ 
- usd - курс доллара по данным Центрального Банка РФ
- mredc - индекс недвижимости MREDC
- imoex_pe - индикатор P/E Мосбиржи
- inv_imoex_pe_bonds10y_delta - риск премия фондового рынка по отношению к долговому ($\frac{1}{P/E} - bonds10y$)
- long_entity (short_entity) - открытые длинные (короткие) позиции по ближайшему фьючерсу MIX юр. лицами 
- long_physic (short_physic) - открытые длинные (короткие) позиции по ближайшему фьючерсу MIX физ. лицами
- {base}_return - относительное изменение значения base, где base это один из вышеописанных типов данных

In [2]:
all_base_cols = [
    'price', 'bonds10y', 'ruonia', 'brent', 'gold', 'usd', 'mredc', 'top3_mean_vol', 
    'imoex_pe', 'long_entity', 'short_entity', 'long_physic', 'short_physic', 'long/short_entity_ratio', 
    'long/short_physic_ratio', 'price_return', 'brent_return', 'gold_return', 'usd_return', 'mredc_return', 
    'top3_mean_vol_return', 'imoex_pe_return', 'long_entity_return', 'short_entity_return', 'long_physic_return', 
    'short_physic_return', 'long/short_entity_ratio_return', 'long/short_physic_ratio_return', 'price_vol_adj_return', 
    'price_vol_adj', 'bonds10y_ruonia_delta', 'inv_imoex_pe_bonds10y_delta'
]

Таблица со сгенерированными признаками состоит из:
- Индикаторов тех. анализа
- Значения скользящих статистик, отношений и разности скользящих статистик с разными окнами

Используемые тех. индикаторы

<table>
  <tr>
    <td><b>Название метода</b></td>
    <td><b>Параметры</b></td>
    <td><b>Источник/Пояснение</b></td>
  </tr>
  <tr>
    <td>rsi</td>
    <td>col<br>windows<br>moving_average_type</td>
    <td><a href="https://en.wikipedia.org/wiki/Relative_strength_index">ссылка</a></td>
  </tr>
  <tr>
    <td>macd</td>
    <td>col<br>short_windows<br>long_windows<br>signal_windows<br>cols_to_use(0,1,2)</td>
    <td><a href="https://en.wikipedia.org/wiki/MACD">ссылка</a></td>
  </tr>
  <tr>
    <td>stochastic_oscillator</td>
    <td>col<br>high_col<br>low_col<br>windows_k<br>windows_n<br>moving_average_type<br>cols_to_use(0,1)</td>
    <td><a href="https://en.wikipedia.org/wiki/Stochastic_oscillator">ссылка</a></td>
  </tr>
  <tr>
    <td>custom_aroon</td>
    <td>col<br>windows<br>cols_to_use(0,1,2)</td>
    <td><a href="https://www.investopedia.com/ask/answers/112814/what-aroon-indicator-formula-and-how-indicator-calculated.asp">ссылка</a></td>
  </tr>
  <tr>
    <td>stat_to_price_ratios</td>
    <td>col<br>stats<br>windows</td>
    <td>Отношение текущей цены к скользящей статистике</td>
  </tr>
  <tr>
    <td>roc</td>
    <td>col<br>windows</td>
    <td>Относительное изменение цены</td>
  </tr>
  <tr>
    <td>bollinger_bands</td>
    <td>col<br>n_sigmas<br>windows<br>cols_to_use(0,1,2)</td>
    <td><a href="https://en.wikipedia.org/wiki/Bollinger_Bands">ссылка</a></td>
  </tr>
  <tr>
    <td>distribution_oscillator (очень долго считает, для одного окна примерно 5 мин)</td>
    <td>cols<br>windows<br>roc_windows</td>
    <td>Считает значения кумулятивной функции распрделения Стьюдента, где сл. величиной является доходность, посчитанная за окно roc_windows, за периоды длиной window. Параметры функции распрделения подбираются в каждой точке при помощи метода fit в <a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_continuous.fit.html#scipy.stats.rv_continuous.fit">scipy</a></td>
  </tr>
</table>

<br><br>
Вспомогательная информация

<table>
    <tr>
        <td><b>Параметры</b></td>
        <td><b>Информация о параметре</b></td>
    </tr>
    <tr>
        <td>содержит "col"</td>
        <td>Название столбцов используемые при генерации тех. индикатора</td>
    </tr>
    <tr>
        <td>содержит "windows"</td>
        <td>Список из рамера окон, за которое будет вестить подсчет статистик, используемых в тех. индикаторе</td>
    </tr>
    <tr>
        <td>moving_average_type</td>
        <td>Тип скользящей статистики - простая или экспоненциальная</td>
    </tr>
    <tr>
        <td>cols_to_use(0,1,2)</td>
        <td>Список из индексов столбцов, которые нужно использовать при моделировании. <br> Например, при генерации custom_aroon на выходе мы получаем таблицу со столбцами ["aroon_up_{window}", "aroon_down_{window}", "aroon_delta_{window}"]. Для целей моделирования мы можем захотеть использовать не все столбцы, cols_to_use позволяет выбрать столбцы, которые нужно использовать, при помощи индекса. В скобках показано какие индексы можно использовать для данного тех индикатора </td>
    </tr>
    <tr>
        <td>n_sigmas</td>
        <td>Кол-во ст. откл. (исп-ся только для bollinger_bands)</td>
    </tr>
    <tr>
        <td>stats</td>
        <td>Виды статистик - min, max, mean, median и др. (исп-ся только для stat_to_price_ratios) </td>
    </tr>
</table>

In [6]:
# пример набора параметров для генерации тех. индикаторов

ta_methods = {
    "rsi": dict(
        col="price", windows=[10, 21, 63, 126], moving_average_type="simple"
    ),
    "stochastic_oscillator": dict(
        col="price",
        high_col="price",
        low_col="price",
        windows_k=[10, 21, 63, 126],
        windows_n=[5, 10, 21, 21],
        moving_average_type="simple",
        cols_to_use=[0],
    ),
    "stat_to_price_ratios": dict(
        col="price",
        stats=["mean", "min", "median", "max"],
        windows=[10, 21, 63, 126, 252],
    ),
    "roc": dict(cols=['price_return'], windows=[10, 21, 63, 126, 252]),
    "custom_aroon": dict(col="price", windows=[126, 252, 504]),
#     "distribution_oscillator": dict(
#         cols=["price_return"], windows=[252, 378], roc_windows=[21, 63, 126, 252] # лучше не исп-ть, будет долго считать
#     ),
}

Подсчет скользящей статистики исходит из информации о:
- Названии столбцов (columns), по которым будут расчитаны статистики
- Названии статистик (attrs), которые будут расчитаны
- Размере окон (windows), за которые считаются статистики
- Списке из пар окон (ratio_windows), по которым будует расчитано отношение и разность статистик
- Типах скользящих статистик (smoothing_types) - простая (sw), экспоненциальная (ew)

In [9]:
# определяем какие базовые признаки мы хотим исп-ть
stat_gen_cols = ['price', 'ruonia', 'imoex_pe', 'gold_return', 'long/short_physic_ratio']

stat_gen_kwargs = dict(
    columns=stat_gen_cols,
    attrs=("mean", "std", "skew", "kurt"),
    windows=(5, 10, 21, 63, 126),
    ratio_windows=((5, 21), (10, 21), (10, 63), (21, 63), (21, 126), (63, 126)),
    smoothing_types=("simple", "exponential"),
)

In [17]:
from portfolio_constructor.feature_generator import data_generator

In [18]:
data_generator_kwargs = dict(
    ta_methods=ta_methods,
    stat_gen_kwargs=stat_gen_kwargs
)
data = data_generator(path='mcftrr.xlsx', **data_generator_kwargs)

Для скользящих статистик названия столбцов сконструированы одиним из след. образов: 
- $\text{{base_col}_{smoothing_type}_{attr}_{window}}$. 

Например, **ruonia_ew_mean_21** - 21 дневная эксп. скользящая средняя ставки RUONIA

- $\text{{base_col}_{smoothing_type}_{attr}_{fast_window}/{slow_window}}$

Например, **gold_return_sw_std_10/21** - отношение 10 и 21 дневной простой скользящей ст. откл. доходности золота

- $\text{{base_col}_{smoothing_type}_{attr}_{fast_window}_{slow_window}}$

Например, **long/short_physic_ratio_sw_skew_63_126** - разность 63 и 126 дневной простой скользящей коэффициента асимметрии отношения открытых длинных и коротких позиций по фьючерсу MIX физ. лицами 

---
Для индикаторов тех. анализа названия зачастую содержат название метода и окно:
- **max/price_252** - отношение максимального значения за 252 скользящее окно и текущего значения
- **aroon_down_252, aroon_down_252, aroon_delta_252** - посчитанный индиктора aroon с окном 252

In [20]:
list(data.columns)

['price',
 'bonds10y',
 'ruonia',
 'top3_mean_vol',
 'imoex_pe',
 'long_entity',
 'short_entity',
 'long_physic',
 'short_physic',
 'long/short_entity_ratio',
 'long/short_physic_ratio',
 'price_return',
 'brent_return',
 'gold_return',
 'usd_return',
 'mredc_return',
 'top3_mean_vol_return',
 'imoex_pe_return',
 'long_entity_return',
 'short_entity_return',
 'long_physic_return',
 'short_physic_return',
 'long/short_entity_ratio_return',
 'long/short_physic_ratio_return',
 'price_vol_adj_return',
 'bonds10y_ruonia_delta',
 'inv_imoex_pe_bonds10y_delta',
 'ruonia_daily',
 'rsi_sw_10',
 'rsi_sw_21',
 'rsi_sw_63',
 'rsi_sw_126',
 'oscillator_sw_10',
 'oscillator_sw_21',
 'oscillator_sw_63',
 'oscillator_sw_126',
 'mean/price_10',
 'min/price_10',
 'median/price_10',
 'max/price_10',
 'mean/price_21',
 'min/price_21',
 'median/price_21',
 'max/price_21',
 'mean/price_63',
 'min/price_63',
 'median/price_63',
 'max/price_63',
 'mean/price_126',
 'min/price_126',
 'median/price_126',
 'max/

После отбора признаков, для построение стратегии нужно опеределить следующие гиперпараметры:

1. Гиперпараметры расширующегося окна
2. Гиперпараметры алгоритма разметки
3. Гиперпараметры алгоритма взвешивания наблюдений
4. Гиперпараметры модели (в нашем случае - Catboost)

**Гиперпараметры расширяющегося окна**

- Изначальное окно (initial_window) - Кол-во точек в тренировочной выборке на первой итрации, по умолчанию равен 500
- Шаг расширения окна (step_length) - Насколько увеличится тренировочная выборка на след. итрации, по умолчанию равен 20
- Горизонт прогнозирования (forecast_horizon) - На сколько точек нужно построить предсказание, по умолчанию 20
- Взвешивание наюблюдений (set_sample_weights) - Флаг для использования алгоритма взвешивания, по умолчанию используется 

Неиспользуемые параметры (не успели проверить идею)
- eval_obs - кол-во точек, извлекаемые из тренировочной выборки с конца, для валидации модели. Исходя из метрик на eval_obs можно выбрать лучшее дерево, которое будет использоваться для предсказания тестовых данных

In [23]:
step = 20
splitter_kwargs = dict(
    forecast_horizon=[i for i in range(1, step + 1)],
    step_length=step,
    initial_window=500,
    window_length=None,
    eval_obs=0,
    set_sample_weights=True,
)

**Гиперпараметры алгоритма разметки**

- Название алгоритма (markup_name) - метод тройного барьера (triple_barrier) или метод локальных максимумов/минимумов (min_max). 

В зависимости от выбранного алгоритма нужна настроить следующие гиперараметры разметки (markup_kwargs). <br>
Если используется метод локальных максимумов/минимумов, то
- Размер окна (freq), в котором отмечаются минимум/максимум для последующего отбора согласно алгоритму
    
Если используется метод тройного барьера, то
- TODO



In [24]:
position_rotator_kwargs_1 = dict(
    markup_name='triple_barrier',
    markup_kwargs=dict(
        h=250, shift_days=63, vol_span=20, volatility_multiplier=2
    )
)

position_rotator_kwargs_2 = dict(
    markup_name='min_max',
    markup_kwargs=dict(
        freq=63
    )
)

**Гиперпараметры алгоритма взвешивания наблюдений**

- Коэффициент свежести (decay_ratio) - определяет, во сколько раз последнее наблюдение важнее первого в рассматриваемом окне (по умолчанию 100)
<br><br>
- Порог значимости движения (shock_quantile) - задает критический квантиль доходности (по умолчанию 1%). Если доходность актива в конкретную дату оказывается ниже этого порога, алгоритм активирует механизм коррекции весов
<br><br>
- Коэффициент шока (shock_factor) - определяет степень увеличения веса для наблюдений, где доходность преодолела порог значимости (по умолчанию 20)
<br><br>
- Коэффициент затухания (shock_decay) - указывает период, в течение которого происходит постепенная нормализация весов после значимого движения (по умолчанию 21 торговых дней)

Демонстрацию алгоритма можно увидеть в файл *sample_weighting_algorithm.ipynb*

In [25]:
# вложенность словарей связана с попытками применения других алгоритмов взвешивания, 
# их можно использовать при моделировании, но мы решили продемонстрировать самый удачный вариант

sample_weight_kwargs = dict(
    weight_params=dict(
        time_critical=dict(
            k=100,
            q=0.01,
            jump_coef=20,
            fading_factor=21
        )
    )
)

**Гиперпараметры модели**

- Название модели (model_name) - название используемой модели. Изначально предполагали написать base класс для различных моделей для целей для целей сравнения и отбора лучшего из них, к сожалению не успели это реализовать, в итоговом варианте используется только catboost
<br><br>
- Кол-во моделей (n_models) - кол-во моделей используемых для предсказания. Простой ансмабль из catboost в котором итоговое предсказание усредняется, модели между собой отличаются только random_state, для каждой из них он определяется как <center> $\text{random_state} = \text{random_state} + \text{n_model}$, где </center>
n_model - порядковый номер модели 
<br><br>
- Остальные параметры (iterations, subsample, random_state) - внутренние гиперпараметры Catboost

In [26]:
model_kwargs = dict(
    model_name="catboost",
    n_models=1,
    iterations=10,
    subsample=0.8,
    random_state=2,
)

Отлично, у нас все готово для прогнозирования и последующего конструирования портфеля 

#### Сначала разделим данные на трейн и тест, подготовим разметку и веса наблюдений

In [57]:
from portfolio_constructor.model import Dataset, Model, Strategy

In [31]:
dataset = Dataset(
    splitter_kwargs,
    sample_weight_kwargs,
    position_rotator_kwargs_1,
)
features = list(data.drop(["price", "ruonia_daily"], axis=1, errors="ignore").columns)
batches = dataset.get_batches(data, features)

Каждый батч состоит из словаря pool и массива дат subset_dates

Pool состоит из:
- X (матрица признаков), 
- y (таргет), 
- weights (веса наблюдений)

subset_dates содержит даты, за которые учитывалась информация, для создания pool

In [39]:
batches[0][0]

{'train': {'data':               price  bonds10y    ruonia  top3_mean_vol  imoex_pe  long_entity  \
  date                                                                            
  2010-01-11  1556.58       NaN       NaN            NaN       NaN          NaN   
  2010-01-12  1538.15       NaN  0.029800   9.479544e+09    7.0543          NaN   
  2010-01-13  1546.06       NaN  0.031000   7.628086e+09    6.9362          NaN   
  2010-01-14  1568.29       NaN  0.034000   8.031201e+09    6.9307          NaN   
  2010-01-15  1565.08       NaN  0.035900   7.009166e+09    6.9822          NaN   
  ...             ...       ...       ...            ...       ...          ...   
  2011-12-30  1563.34    0.0782  0.048600   5.417504e+09    4.2201          NaN   
  2012-01-03  1610.75    0.0780  0.048114   3.574828e+09    4.2766          NaN   
  2012-01-04  1613.91    0.0780  0.047629   4.245687e+09    4.5963          NaN   
  2012-01-05  1599.77    0.0784  0.047143   5.673685e+09    4.5963    

#### Обучим модели на полученных батчах, кол-во батчей эквивалетно кол-ву моделей которые будут обучены

In [40]:
model = Model(
    dataset,
    model_kwargs,
)
preds, train_info = model.get_predictions(batches)

batches:  39%|███████████████████████████▊                                            | 64/166 [01:02<01:46,  1.05s/it]


batches:  78%|███████████████████████████████████████████████████████▌               | 130/166 [02:29<00:50,  1.41s/it]


total_models: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [03:22<00:00, 202.84s/it]


На выходе получаем preds и train_info

В preds записан рез-тат predict_proba из Сatboost (возвращает "вероятность" таргета = 1, т.е. находится ли в предсказанную дату временной ряд на восходящем тренде)

In [50]:
preds.head()

,preds_seed_2
2012-01-09,0.998669
2012-01-10,0.998564
2012-01-11,0.994792
2012-01-12,0.996227
2012-01-13,0.987779


в train_info записаны рез-таты обучения каждой из модели, содержащая:
- Даты начала и конца тестового периода (from_date, till_date)
- Среднее значение таргета на трейне (mean_train_target)
- Среднее значение таргета на тесте (mean_test_target)
- Вовзращаемые моделью значения feature_importnace (значения из списка features)
- Порядковый номер модели (n_model)

In [49]:
train_info.head()

,from_date,till_date,mean_train_target,mean_test_target,price,bonds10y,ruonia,top3_mean_vol,imoex_pe,long_entity,...,ruonia_sw_kurt_63/126,imoex_pe_sw_kurt_63/126,gold_return_sw_kurt_63/126,long/short_physic_ratio_sw_kurt_63/126,price_sw_kurt_63_126,ruonia_sw_kurt_63_126,imoex_pe_sw_kurt_63_126,gold_return_sw_kurt_63_126,long/short_physic_ratio_sw_kurt_63_126,n_model
0,2012-01-09,2012-02-03,0.646000,0.0,0.193455,0.0,0.016739,0.0,0.0,0.0,...,0.000000,0.0,0.010014,0.0,0.0,0.0,0.0,0.000000,0.0,1
1,2012-02-06,2012-03-05,0.523077,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.644298,0.0,0.000000,0.0,0.0,0.0,0.0,0.539880,0.0,1
2,2012-03-06,2012-04-03,0.503704,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,1.074854,0.0,1
3,2012-04-04,2012-05-02,0.485714,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.302294,0.0,1
4,2012-05-03,2012-05-29,0.468966,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.294316,0.0,1


#### Строим портфель и смотрим на рез-тат :)

In [61]:
strategy = Strategy(
    model,
    prob_to_weight=True # не округляет вероятность                      
)

strat_data = data.loc[:, ['price', 'price_return', 'ruonia', 'ruonia_daily']].copy()
strat_kwargs = dict(
    perf_plot=True, sliding_plot=True # сразу рисует динамику портфеля
)
output = strategy.base_strategy_peformance(strat_data, preds, **strat_kwargs)

На нижнем графике показано какой PnL получил бы инвестор, если бы он инвестировал в портфель или в индекс MCFTRR 2 года назад.

In [63]:
# основные метрики портфеля
output['metrics']

{'strategy_perf': 791.7674818085734,
 'bench_perf': 394.1267177368901,
 'mean_outperf': 69.14293603409838,
 'sharpe_ratio_rf': 0.037423416300926916,
 'sharpe_ratio_rm': 0.016276103371840857,
 'weighted_sharpe_ratio_rf': 0.03920443351362568,
 'weighted_sharpe_ratio_rm': 0.018956452921622503,
 'max_drawdown': -0.33004143892662174,
 'max_recovery': 326.0,
 'beta': 0.3902148827551663,
 'var': -0.023010462720213856,
 'cvar': -0.036584002223303484}

In [68]:
# динамика индексов доходности портфеля и MCFTRR
output['res'].loc[:, ['strategy_perf', 'bench_perf']]

,strategy_perf,bench_perf
date,,
2012-01-10,100.000000,100.000000
2012-01-11,101.965889,101.968701
2012-01-12,101.182342,101.180973
2012-01-13,101.117603,101.115948
2012-01-16,101.043367,101.040396
...,...,...
2025-03-31,836.160130,417.015426
2025-04-01,832.873352,415.321687
2025-04-02,819.803414,408.577692
